# 4.2. Band-Specific Oscillatory Analysis

**Learning Objectives:**
- Understand the functional significance of different frequency bands
- Extract and analyze band-specific power
- Explore event-related desynchronization (ERD) and synchronization (ERS)
- Compare oscillatory patterns across brain regions
- Relate oscillatory changes to cognitive processes
- Perform statistical analysis of band power

**Key Concepts:**
- **Frequency Bands**: Delta, theta, alpha, beta, gamma and their roles
- **ERD (Event-Related Desynchronization)**: Power decrease indicating active processing
- **ERS (Event-Related Synchronization)**: Power increase indicating inhibition or specific computation
- **Mu Rhythm**: Sensorimotor alpha rhythm (8-12 Hz over motor cortex)
- **Functional Specificity**: Different bands subserve different cognitive functions

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, stats
import pandas as pd
import mne
from pathlib import Path
from mne.time_frequency import tfr_morlet, psd_array_multitaper
from mne.datasets import sample
from mne.stats import permutation_cluster_1samp_test

import warnings
warnings.filterwarnings('ignore')

In [2]:
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.1)
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

### 1. Frequency Bands and Their Functions

Brain oscillations are classified into frequency bands, each associated with distinct cognitive and physiological states:

| Band | Frequency | Primary Functions | When It Appears |
|------|-----------|-------------------|-----------------|
| **Delta (δ)** | 0.5-4 Hz | Deep sleep, unconsciousness, slow-wave sleep | Sleep stages 3-4 |
| **Theta (θ)** | 4-8 Hz | Memory encoding/retrieval, drowsiness, meditation, creativity | REM sleep, relaxed wakefulness |
| **Alpha (α)** | 8-12 Hz | Relaxed wakefulness, inhibition, idling state | Eyes closed, relaxed |
| **Mu (μ)** | 8-13 Hz | Motor cortex idling (similar to alpha but motor-specific) | Rest, no movement |
| **Beta (β)** | 13-30 Hz | Active thinking, focus, motor control, anxiety | Active tasks, concentration |
| **Gamma (γ)** | 30-100 Hz | Attention, binding, cognitive processing, consciousness | Active perception/cognition |

### Key Principles

1. **ERD (Desynchronization)**: Power **decrease** → Active processing, engagement
   - Example: Alpha ERD during eyes opening (visual cortex active)
   - Example: Beta ERD during movement preparation (motor cortex active)

2. **ERS (Synchronization)**: Power **increase** → Inhibition or specific computation
   - Example: Alpha ERS when closing eyes (visual cortex idling)
   - Example: Beta rebound after movement (motor cortex resetting)

3. **Regional Specificity**: Different rhythms dominate in different brain areas
   - Occipital alpha: Visual processing
   - Central mu: Motor control
   - Frontal theta: Executive function

### 2. Load and Prepare Data

We'll use the MNE sample dataset with auditory and visual stimuli and perform basic preprocessing steps.

In [4]:
repo_root = Path("./").resolve().parents[1]
target_dir = repo_root / "datasets" / "meg_sample"
target_dir.mkdir(parents=True, exist_ok=True)

data_path = sample.data_path(path=str(target_dir), download=True)
meg_file = Path(data_path) / "MEG" / "sample" / "sample_audvis_raw.fif"
events_fname = data_path / 'MEG' / 'sample' / 'sample_audvis_raw-eve.fif'

raw = mne.io.read_raw_fif(meg_file, preload=True)
events = mne.read_events(events_fname)

# Focus on EEG channels 
raw.pick_types(meg=False, eeg=True, eog=False, stim=False)

print(f"Channels: {raw.ch_names[:5]}... and {raw.ch_names[-1]}")

Opening raw data file /Users/yibeisita/Documents/neuro-ai-playground/datasets/meg_sample/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Channels: ['EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005']... and EEG 060


In [5]:
# Apply bandpass filter (1-100 Hz to preserve all bands)
raw.filter(l_freq=1.0, h_freq=100.0, verbose=False)

<Raw | sample_audvis_raw.fif, 59 x 166800 (277.7 s), ~78.0 MiB, data loaded>

In [6]:
# Event dictionary
event_id = {
    'auditory/left': 1, 
    'auditory/right': 2,
    'visual/left': 3, 
    'visual/right': 4
}

In [7]:
# Create epochs
epochs = mne.Epochs(
    raw, events, event_id,
    tmin=-1.0, tmax=2.0,  # Longer window to see post-stimulus effects
    baseline=(-1.0, -0.5),  # Early baseline to avoid anticipation
    preload=True,
    verbose=False
)

In [9]:
print(f"Loaded {len(epochs)} epochs")
print(f"Sampling rate: {epochs.info['sfreq']} Hz")
print(f"Number of channels: {len(epochs.ch_names)}")
print(f"\nEvent counts:")

for event_name, count in zip(event_id.keys(), [len(epochs[e]) for e in event_id.keys()]):
    print(f"- {event_name}: {count}")

Loaded 289 epochs
Sampling rate: 600.614990234375 Hz
Number of channels: 59

Event counts:
- auditory/left: 72
- auditory/right: 73
- visual/left: 73
- visual/right: 71


### 3. Define Frequency Bands

Let's define our frequency bands with more granular subdivisions for detailed analysis.

In [11]:
# Define frequency bands
BANDS = {
    # Standard bands
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (13, 30),
    'Low Gamma': (30, 50),
    'High Gamma': (50, 100),
    
    # Subdivisions for detailed analysis
    'Low Alpha': (8, 10),
    'High Alpha': (10, 12),
    'Low Beta': (13, 20),
    'High Beta': (20, 30),
}

BAND_COLORS = {
    'Delta': '#1f77b4',
    'Theta': '#ff7f0e',
    'Alpha': '#2ca02c',
    'Beta': '#d62728',
    'Low Gamma': '#9467bd',
    'High Gamma': '#8c564b',
    'Low Alpha': '#98df8a',
    'High Alpha': '#2ca02c',
    'Low Beta': '#ff9896',
    'High Beta': '#d62728',
}

print("Defined Frequency Bands:")
print("-" * 50)
for band_name, (fmin, fmax) in BANDS.items():
    print(f"{band_name:15s}: {fmin:4.1f} - {fmax:5.1f} Hz")

Defined Frequency Bands:
--------------------------------------------------
Delta          :  1.0 -   4.0 Hz
Theta          :  4.0 -   8.0 Hz
Alpha          :  8.0 -  12.0 Hz
Beta           : 13.0 -  30.0 Hz
Low Gamma      : 30.0 -  50.0 Hz
High Gamma     : 50.0 - 100.0 Hz
Low Alpha      :  8.0 -  10.0 Hz
High Alpha     : 10.0 -  12.0 Hz
Low Beta       : 13.0 -  20.0 Hz
High Beta      : 20.0 -  30.0 Hz


### 4. Extract Band Power Using Filtering

One approach to band-specific analysis is to filter the signal into different frequency bands and compute power.

In [13]:
def compute_band_power(epochs, band_name, fmin, fmax):
    """
    Filter epochs to a specific band and compute power.
    """
    # Filter to the band
    epochs_band = epochs.copy().filter(
        l_freq=fmin, h_freq=fmax,
        verbose=False
    )
    
    # Get data and compute power (squared amplitude)
    data = epochs_band.get_data()
    power = data ** 2
    
    return power # Power time series (n_epochs, n_channels, n_times)

In [ ]:
# Example: Compute alpha power
alpha_power = compute_band_power(epochs, 'Alpha', 8, 12)

print(f"Alpha power shape: {alpha_power.shape}")
print(f"(n_epochs={alpha_power.shape[0]}, n_channels={alpha_power.shape[1]}, n_times={alpha_power.shape[2]})")

Alpha power shape: (289, 59, 1803)
(n_epochs=289, n_channels=59, n_times=1803)
